In [2]:
import pickle
import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
import copy

In [3]:
# 파일과 필요한 데이터 지정 #
path = 'jobplanet_crawling.csv'
selected_columns = ['title','positive','negative','advice','star_score']

In [6]:
# 수집한 데이터 불러오기 #
def load_dataset(path):
    all_df = pd.read_csv(path, encoding='utf-8')
    # with open(path, 'rb') as f:
    #     all_df = pickle.load(f)
    return all_df
# 데이터프레임에서 원하는 컬럼만 선택 #
def select_columns(all_df, columns):
    return all_df[columns]
# 데이터프레임의 컬럼 타입 변경 #
def define_dtype(df):
    df['star_score'] = df['star_score'].astype(int) # type 변경
    df['star_score'] = np.where(df['star_score'] == 10, 100, df['star_score']) 
    #크롤링 시 100점이 10점으로 잘려 수집되었기 때문에 다시 100점으로 변경
# 긍정, 부정 리뷰를 하나의 데이터프레임으로 만들기 #
def make_reviews(df):
    positive = df['positive'].tolist()
    negative = df['negative'].tolist()
    pos_label = np.ones(len(df['positive'].tolist()), dtype=int) 
    neg_label = np.zeros(len(df['negative'].tolist()), dtype=int )
    text = positive + negative
    label = np.concatenate([pos_label, neg_label])
    reviews = pd.DataFrame({'text':text, 'label':label})
    return reviews
# dataloader 만들기 #
def dataloader(pat, selected_columns):
    all_df = load_dataset(path)
    df = select_columns(all_df, selected_columns)
    define_dtype(df)
    reviews = make_reviews(df)
    return reviews

In [7]:
reviews = dataloader(path, selected_columns)
reviews.head()

C:\Users\user\AppData\Local\Temp/ipykernel_20636/3567241950.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_score'] = df['star_score'].astype(int) # type 변경
C:\Users\user\AppData\Local\Temp/ipykernel_20636/3567241950.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_score'] = np.where(df['star_score'] == 10, 100, df['star_score'])


,text,label
0,"테이크아웃, 셔틀, 인지도, 보너스(많이나올때기준), 적당한 인성의 동료들",1
1,타기업대비 뚜렷한 장점은 없고 밥맛은 괜찮아요 삼시세끼주는것도,1
2,돈을 많이 주고 성과금도 나름 괜찮은 편이라 돈보고 일하긴 좋다,1
3,네임벨류가 좋음\n복지는 중간정도? 경쟁사 복지를 따라하는척하긴함,1
4,"대외적 인식이 좋음 (특히 노년층기준), ps뽕, 반도체 기준 아무리 먼 온양도 지...",1


In [8]:
p = re.compile('\n')
# \n 제거 #
reviews['text'] = reviews['text'].apply(lambda x: p.sub('', x))
#띄어쓰기를 위한 공백 제거 #
reviews['text'] = reviews['text'].apply(lambda x: x.replace(' ', ''))

In [9]:
reviews['text']

0                      테이크아웃,셔틀,인지도,보너스(많이나올때기준),적당한인성의동료들
1                             타기업대비뚜렷한장점은없고밥맛은괜찮아요삼시세끼주는것도
2                               돈을많이주고성과금도나름괜찮은편이라돈보고일하긴좋다
3                            네임벨류가좋음복지는중간정도?경쟁사복지를따라하는척하긴함
4        대외적인식이좋음(특히노년층기준),ps뽕,반도체기준아무리먼온양도지하철이닿기는닿음(서울...
                               ...                        
14995                         툭히콘텐츠계열에서는소통이자유로워야하는데그부분이아쉬움
14996                            타대기업에비해처우가부족한면이있고관료적인면이있음
14997           일이빡쌔요.워라벨이보장되긴하지만일이많이힘들고고단함열심히하면워라밸보장되니좋아요
14998                    상하관계에너무얽메겨있음일하는데있어어는좀번거로울수도있다는느낌.
14999                              일많은부서로가면같은월급인데야근많이할수도있음
Name: text, Length: 15000, dtype: object

In [21]:
from pykospacing import Spacing
space = Spacing()
# corpus = [space(text) for text in corpus] #20m
reviews['text'] = reviews['text'].apply(lambda x: space(x)) #54m

In [26]:
corpus = reviews['text']

In [65]:
# 맞춤법 교정 #
def spell_checker(text):
    import requests
    import json
    # 1. 텍스트 준비 & 개행문자 처리 #
    text = text.replace('\n', '\r\n')
    # 2. 맞춤법 검사 요청 (requests) #
    response = requests.post('http://164.125.7.61/speller/results', data={'text1': text})
    # 3. 응답에서 필요한 내용 추출 (html 파싱) #
    data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] 
    # 4. 파이썬 딕셔너리 형식으로 변환 #
    data = json.loads(data)
    # 5. 교정 내용 출력 # 
    # print('original: ', data['str'])
    for i in range(len(data['errInfo'])):
        if '|' in data['errInfo'][i]['candWord']:
            # '|'로 구분된 후보군 중 첫번째 단어 선택 #
            data['errInfo'][i]['candWord'] = data['errInfo'][i]['candWord'].split('|')[0] 
        data['str'] = data['str'].replace(data['errInfo'][i]['orgStr'],data['errInfo'][i]['candWord'])
    return data['str']
    

In [66]:
# 맞춤법 교정 #
for i in range(len(corpus)):
    try:
        corpus[i] = spell_checker(corpus[i])
    except:
        continue

C:\Users\user\AppData\Local\Temp/ipykernel_12384/3763769364.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus[i] = spell_checker(corpus[i])


In [ ]:
# # Checkpoint #
# reviews['text'] = corpus
# with open('reviews_spell_check.pickle', 'wb') as f:
#     pickle.dump(reviews, f)

# with open('reviews_spell_check.pickle', 'rb') as f:
#     reviews = pickle.load(f)

# csv로 저장 #
# reviews.to_csv('reviews_spell_check.csv', index=False, encoding='utf-8-sig')

### 중간정리 ###
- 15000여개의 기업 리뷰(긍정:7500,부정:7500개)를 수집
- 특수문자 제거, 토큰화 작업 전에 띄어쓰기와 맞춤법 교정완료
- 특수문자 제거, 정규화, 토큰화
- 품사태깅할 때 : 불용어 제거, Lemmatize/stemming, 

In [30]:
reviews = pd.read_csv('reviews_spell_check.csv', encoding='utf-8-sig')

In [32]:
# 특수문자 제거 #
def remove_special(text):
    text = re.sub('[^가-힣a-zA-Z0-9 ]', '', text)
    return text

In [33]:
# 정규화 #
okt = Okt()
reviews['text'] = reviews['text'].apply(lambda x: okt.normalize(x))
reviews

,text,label
0,"테이크아웃, 셔틀, 인지도, 보너스(많이 나올 때 기준), 적당한 인성의 동료들",1
1,타 기업 대비 뚜렷한 장점은 없고 밥맛은 괜찮아요. 삼시세끼 주는 것도,1
2,돈을 많이 주고 성과금도 나름 괜찮은 편이라 돈 보고 일하긴 좋다,1
3,네임벨류가 좋음. 복지는 중간 정도? 경쟁사 복지를 따라 하는 척하긴 함,1
4,"대외적인 식이 좋음(특히 노년층 기준), PS 뽕, 반도체 기준 아무리 먼 온양도 ...",1
...,...,...
14995,특히 콘텐츠 계열에서는 소통이 자유로워야 하는데 그 부분이 아쉬움,0
14996,타 대기업에 비해 처우가 부족한 면이 있고 관료적인 면이 있음,0
14997,일이 거세요. 일과 삶의 균형이 보장되긴 하지만 일이 아주 힘들고 고단함. 열심히 ...,0
14998,상하 관계에 너무 얽메겨 있음 일하는 데 있어 오는 좀 번거로울 수도 있다는 느낌.,0


In [34]:
# 특수문자 제거 # 
reviews['text'] = reviews['text'].apply(lambda x: remove_special(x))

In [35]:
# 토큰화 및 불용어 제거 #
with open('korean_stopwords.txt','r',) as f:
    stopwords = f.readlines()
    stopwords = list(map(lambda s: s.strip(), stopwords))
    
# 불용어 추가(Additional Task) #
# 서술격조사, 용언 접사(하다) 불용어 추가 # 
stopwords.extend(['하다','이다','있다', '없다', '많다', '되다', '회사', '하고'])

def tokenize(text):
    tokens = okt.morphs(text, stem=True)
    tokens = [token for token in tokens if token not in stopwords and len(token) > 1]
    return tokens

reviews['text'] = reviews['text'].apply(lambda x: tokenize(x))

In [36]:
# 특정 단어가 포함된 행 개수 세기 #
def count_exist(text,char):
    count = 0
    char = char
    for i in range(len(text)):
        if char in text[i]:
            count += 1
    return count
char = '없다'
reviews['count_exist'] = reviews['text'].apply(lambda x: count_exist(x,char))
reviews.groupby('label')['count_exist'].sum()

label
0    178
1     48
Name: count_exist, dtype: int64

In [37]:
# 문자열로 합치기 #
reviews['text'] = reviews['text'].apply(lambda x : ' '.join(x))

In [38]:
reviews = reviews[['text','label']]

In [44]:
# 예외처리 #
# 공백값이 있다면 특수토큰으로 대체 # 
reviews['text'] = np.where(reviews['text'] == '', '999999999', reviews['text'])

C:\Users\user\AppData\Local\Temp/ipykernel_10620/581257447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['text'] = np.where(reviews['text'] == '', '999999999', reviews['text'])


In [46]:
# csv로 저장 #
reviews.to_csv('reviews_tokenized.csv', index=False, encoding='utf-8-sig')